In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
# Import modules
import numpy as np # maths
from scipy.interpolate import interp1d
from scipy.optimize import root_scalar

from time import time# timer for debug

    # gather results obtained from the runs
import pickle # open .pkl files where python objects have been saved
from safe import safe # the empty class which is used to save the results
    
    # system commands
import os
    
    # interact with the C++ core and the Python core
import dimers as dim # C++ interface module
import KagomeFunctions as kf # "library" allowing to work on Kagome
import KagomeDrawing as kdraw # "library" allowing to plot Kagome
import Observables as obs #observables that can be used by the run script
import KagomeFT as kft #small functions to compute the FT
import AnalysisFunctions as af # functions to make the analysis code cleaner
    #plots using matplotlib
import matplotlib
import matplotlib.pyplot as plt


%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams.update({'font.size': 14, 'pgf.texsystem':'pdflatex'})

In [ ]:
# This table gives the expression of the G.S. energy:
# E_GS = -Tbl[0]*h + Tbl[1]*J1+ Tbl[2]*J2 + Tbl[3]*J3||
# One can thus deduce the magnetisation and correlations
mline = 2/9;
TNvalsTable = {"Zero field":[0, -1/3, -1/3, 1/3],
               "Small field":[1/9, -1/3 - 1/81, -1/3-1/81, 1/3-1/81, 5/9-1/81],
               "Line":[mline, -1/3 - mline**2, -1/3-mline**2],
               "Plateau":[1/3, -1/3-1/9, -1/3-1/9, 1 -1/9, 1-1/9],
               "Large fied":[],
               "Saturation":[1, 2, 2, 2]
              }


In [ ]:
# Create a folder to save the pictures
foldername = '../../Analysis_PSI_EPFL/Runs_25-09-20_J1J2J3pField/'
foldername = '../../Analysis_PSI_EPFL/Runs_28-09-20_J1J2J3p_LargeField/'
#foldername = '../../Analysis_PSI_EPFL/Runs_28-09-20_J1J2J3p_VerySmallField/'
#foldername = '../../Analysis_PSI_EPFL/Runs_28-09-20_J1J2J3p_SmallField/'
foldername = '../../Analysis_PSI_EPFL/Runs_05-10-20_J1h/'
#foldername = '../../Analysis_PSI_EPFL/Runs_16-11-20_J1J2J3p_LargeField/'
#foldername = '../../Analysis_PSI_EPFL/Runs_08-12-20_J1J2J3pLine/'
#foldername = '../../Analysis_PSI_EPFL/Runs_04-01-21_J1J2J3pLine/'


results_foldername = 'Results_07-01-21'


filenamelist = ['KJ1Field_L4_MEASFIELD_nsm4096_ncores9_J20_J30_J3st0_folder/backup'
                #'KJ1Field_L4_MEASFIELD_J1noh_nsm4096_ncores9_J20_J30_J3st0_folder/backup'
                #'KJ1J2J3pField_MEASFIELD_L4_nsm4096_ncores9_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_MEASFIELD_L4_nsm4096_ncores9_J20.0233_J30_J3st0_folder/backup'
                #'KJ1J2J3pField_SMALLFIELD_L4_nsm8192_ncores6_J20.0233_J30_J3st0_folder/backup'
                #'KJ1J2J3pField_SMALLFIELD_L4_nsm8192_ncores9_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_LARGEFIELD_L4_nsm8192_ncores6_J20.0233_J30_J3st0_folder/backup'
                 #'KJ1J2J3pField_MEDIUM_LARGEFIELD_L4_nsm8192_ncores9_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_SHORT_LARGEFIELD_L4_nsm512_ncores9_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_VERYSMALLFIELD_L4_nsm1024_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_MED_VERYSMALLFIELD_L4_nsm8192_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_LONG_RES_VERYSMALLFIELD_L4_nsm32768_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_LONG_RES_VERYSMALLFIELD_L8_nsm32768_ncores8_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_SMALLFIELD_L4_nsm1024_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_MEDFIELD_L4_nsm1024_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_MEDFIELD2_L4_nsm1024_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_MED_SMALLFIELD_L4_nsm8192_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_LONG_RES_LARGEFIELD_L4_nsm16384_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_VERYSMALLFIELD_L8_nsm1024_ncores4_J20.0233_J30.0102_J3st0_folder/backup'
                #'KJ1J2J3pField_MEDIUM_LARGEFIELD_L4_nsm4096_ncores8_J20.2_J30.1_J3st0_folder/backup'
                #'KJ1J2J3ph_PTLine_L4_J20.02_J3p0.01_restarted1_folder/backup'
                #'KJ1J2J3ph_PTLine_L4_J20.02_J3p0.01_restarted2_folder/backup'
                #'KJ1J2J3ph_PTLine_L8_J20.02_J3p0.01_it0_nsm1024_Started_folder/backup'
]

os.makedirs('./' + foldername + results_foldername, exist_ok = True)

n = len(filenamelist)

In [ ]:
[L, numsites, J1, J2, J3, J3st, J4, nb, num_in_bin, 
 htip, Ttip, pswitch, uponly, path,
 temperatures, nt,
 stat_temps, temperatures_plots, hfields, nh,
 stat_hfields, hfields_plots, listfunctions, sref, idswalker] = \
af.LoadParameters(foldername, filenamelist)

print(J3)
print(J4)
s_ijl, ijl_s = kdraw.createspinsitetable(L[0])
print("nb ", nb, "num_in_bin ", num_in_bin)
swapst_th, swapsh_th, swapst, swapsh = af.LoadSwaps(foldername, filenamelist, nb, num_in_bin, nh, nt)

n = len(L)
failedth, failedssfth, failed, failedssf = \
af.LoadUpdates(foldername, filenamelist, nb, num_in_bin, [9*L[i]**2 for i in range(n)])

kw = {'binning': False, 'plzplot': False, 'plotmin': 0, 'plotmax': 17, 'binning_level':0}
[t_h_MeanE, t_h_MeanEsq, t_h_varMeanE, t_h_varMeanEsq,  t_h_VarE, t_h_ErrVarE,C, ErrC,\
            t_h_S, t_h_Smin, t_h_Smax, MergedData] = \
 af.LoadEnergy(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

[t_h_MeanM, t_h_MeanMsq, t_h_varMeanM, t_h_varMeanMsq, Chi, ErrChi] = \
 af.LoadMagnetisation(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

In [ ]:
hfields_plots

In [ ]:
###########################################
        ## ALGORITHM ANALYSIS ##
###########################################
tidmin = 0
tidmax = nt
af.SwapsAnalysis(L, n, tidmin, tidmax, temperatures, hfields,
                 foldername, results_foldername, swapst, swapsh)

In [ ]:
af.FailedAnalysis(L, n, tidmin, tidmax, temperatures, hfields,
                 foldername, results_foldername,failed, failedssf)

In [ ]:
###########################################
tidmin = 0
tidmax = [len(stat_temps[i]) for i in range(n)]
temperatures_plots = np.array(temperatures_plots)
### Energy

In [ ]:
hfields_plots

In [ ]:
import importlib
importlib.reload(af)

In [ ]:
for i in range(0,nh[0]):
    print("h/J1 = ", hfields_plots[0][i], " -> E = ", t_h_MeanE[0][0,i])

In [ ]:
############################################
#        ## Energy ANALYSIS ##
############################################
### Energy
S0 = np.log(2)
kw = {'gscheck': False, 'ploth' : False}
tidmin = 0
for i in range(n):
    af.BasicPlotsE(L, i, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                    results_foldername, filenamelist, t_h_MeanE, t_h_MeanEsq, t_h_varMeanE,
                    t_h_varMeanEsq, C, ErrC, J1, J2, J3, J4, **kw)
plt.show()

In [ ]:
TNvals = TNvalsTable["Line"]

In [ ]:
############################################
#        ## MAGNETISATION ANALYSIS ##
############################################
kw = {'TNvals': TNvals, 'ploth' : False}
for i in range(n):
    af.BasicPlotsM(L, i, 0, tidmax, temperatures_plots, hfields_plots, foldername,
                    results_foldername, filenamelist, t_h_MeanM, t_h_MeanMsq, t_h_varMeanM,
                    t_h_varMeanMsq, Chi, ErrChi, J1, J2, J3, J4, **kw)


    plt.legend()
plt.show()

In [ ]:
rmmag = True
kw = {'rmmag':rmmag,'binning': True, 'plzplot':False}

In [ ]:
t_h_MeanFc, t_h_varMeanFc= \
 af.LoadFirstCorrelations(foldername, filenamelist, listfunctions,stat_temps,
                          stat_hfields, nb,t_h_MeanM,t_h_varMeanMsq, **kw)

In [ ]:
importlib.reload(af)

In [ ]:
t_h_MeanFc[0][1]

In [ ]:
TNvals

In [ ]:
for i in range(n):
     
    for hid in range(nh[i]):
        kw = {'TNvals':TNvals}
        addsave = "L={0} ".format(L[i]) + "J2ovJ1" +"={0}".format(J2[i]/J1[i]) + "J3povJ1" +"={0}".format(J3[i]/J1[i])  + "J3sovJ1" +"={0}".format(J3st[i]/J1[i])
        addtitle = r"$L$={0}".format(L[i])+\
        r"  $J_2/J_1$={0}".format(J2[i]/J1[i]) +\
        r"  $J_{3||}/J_1$" + "={0}".format(J3[i]/J1[i]) +\
        r"  $J_{3\star}/J_1$"+"={0}".format(J3st[i]/J1[i]) +\
        r"  $h/J_1$"+"={0}".format(hfields_plots[i][hid]/J1[i])
   
        af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                       t_h_varMeanFc, foldername, 
                                       results_foldername, filenamelist,
                                       tmin = 0, setyticks = np.arange(-0.4,0.7,0.1),
                                       addsave = addsave,
                                       addtitle = addtitle, hid = hid, **kw)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (11,9), dpi = 200)
markersize = 4
af.BasicPlotsFirstCorrelations(L, 0, t_h_MeanFc, temperatures_plots, 
                                   t_h_varMeanFc, foldername, 
                                   results_foldername, filenamelist,
                                   tmin = 0, setyticks = np.arange(-0.4,0.7,0.1),
                                   addsave = addsave,
                                   addtitle = addtitle, createfig = False, ax = ax, markersize = markersize)

In [ ]:
# Introducing the experimental values for the plots below:
## <sisj>-<si><sj>:
NN1exp = -0.218;
NN1experr = 0.005;
NN1expdev = 0.02;

NN2exp = 0.021;
NN2experr = 0.004;
NN2expdev = 0.02;

NN3pexp = 0.063;
NN3pexperr = 0.005;
NN3pexpdev = 0.02;

NN3sexp = -0.003;
NN3sexperr = 0.005;
NN3sexpdev = 0.02
mexp = -0.193;

rmexpmag = True

if not rmexpmag:
    NN1exp += mexp**2;
    NN2exp += mexp**2;
    NN3pexp += mexp**2;
    NN3sexp += mexp**2;
    

print(NN1exp)
print(NN2exp)
print(NN3pexp)
print(NN3sexp)

In [ ]:
import importlib
importlib.reload(af)

In [ ]:
for i in range(n):
    for hid,h in enumerate(hfields_plots[0]):
        addsave = "L={0}".format(L[i]) + "h={0}".format(h)+"J2ovJ1" +"={0}".format(J2[i]/J1[i]) + "J3povJ1" +"={0}".format(J3[i]/J1[i])  + "J3sovJ1" +"={0}".format(J3st[i]/J1[i])
        addtitle = r"$L$={0} ".format(L[i])+ r", $h/J_1$ = {0} ".format(h)+ r", $J_2/J_1$={0}".format(J2[i]/J1[i]) + r"  $J_{3||}/J_1$" + "={0}".format(J3[i]/J1[i]) +r"  $J_{3\star}/J_1$"+"={0}".format(J3st[i]/J1[i])
        af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                       t_h_varMeanFc, foldername, 
                                       results_foldername, filenamelist,
                                       tmin = 0, setyticks = np.arange(-0.3,0.7,0.1),
                                       addtitle = addtitle,markersize=4,
                                       save = False, hid = hid)
        plt.gca().set_prop_cycle(None)
        Tmin = min(temperatures_plots[i])
        Tmax = max(temperatures_plots[i])
        plt.fill_between([Tmin,Tmax],[NN1exp-NN1expdev,NN1exp-NN1expdev],[NN1exp+NN1expdev, NN1exp+NN1expdev], alpha = 0.2, label = r'$c_1^{E}$')
        plt.fill_between([Tmin,Tmax],[NN2exp-NN2expdev,NN2exp-NN2expdev],[NN2exp+NN2expdev, NN2exp+NN2expdev], alpha = 0.2,  label = r'$c_2^{E}$')
        plt.fill_between([Tmin,Tmax],[NN3pexp-NN3pexpdev,NN3pexp-NN3pexpdev],[NN3pexp+NN3pexpdev, NN3pexp+NN3pexpdev], alpha = 0.2,  label = r'$c_{3||}^{E}$')
        plt.fill_between([Tmin,Tmax],[NN3sexp-NN3sexpdev, NN3sexp-NN3sexpdev],[NN3sexp+NN3sexpdev, NN3sexp+NN3sexpdev], alpha = 0.2,label = r'$c_{3\star}^{E}$')
        plt.xlim([temperatures_plots[0][0], 40])
        plt.ylim([-0.5, 1])
        plt.yticks(np.arange(-0.5,1,0.1))
        plt.legend(loc = 'lower left')
        plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Zoom" + addsave +"rmexpmag={0}.png".format(rmexpmag))

In [ ]:
for i in range(n):
    for hid,h in enumerate(hfields_plots[0]):
        addsave = "L={0}".format(L[i]) + "h={0}".format(h)+"J2ovJ1" +"={0}".format(J2[i]/J1[i]) + "J3povJ1" +"={0}".format(J3[i]/J1[i])  + "J3sovJ1" +"={0}".format(J3st[i]/J1[i])
        addtitle = r"$L$={0} ".format(L[i])+ r", $h/J_1$ = {0} ".format(h)+ r", $J_2/J_1$={0}".format(J2[i]/J1[i]) + r"  $J_{3||}/J_1$" + "={0}".format(J3[i]/J1[i]) +r"  $J_{3\star}/J_1$"+"={0}".format(J3st[i]/J1[i])
        af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                       t_h_varMeanFc, foldername, 
                                       results_foldername, filenamelist,
                                       tmin = 0, setyticks = np.arange(-0.3,0.7,0.1),
                                       addtitle = addtitle,markersize=4,
                                       save = False, hid = hid, log = False)
        plt.gca().set_prop_cycle(None)
        Tmin = min(temperatures_plots[i])
        Tmax = max(temperatures_plots[i])
        plt.fill_between([Tmin,Tmax],[NN1exp-NN1expdev,NN1exp-NN1expdev],[NN1exp+NN1expdev, NN1exp+NN1expdev], alpha = 0.2, label = r'$c_1^{E}$')
        plt.fill_between([Tmin,Tmax],[NN2exp-NN2expdev,NN2exp-NN2expdev],[NN2exp+NN2expdev, NN2exp+NN2expdev], alpha = 0.2,  label = r'$c_2^{E}$')
        plt.fill_between([Tmin,Tmax],[NN3pexp-NN3pexpdev,NN3pexp-NN3pexpdev],[NN3pexp+NN3pexpdev, NN3pexp+NN3pexpdev], alpha = 0.2,  label = r'$c_{3||}^{E}$')
        plt.fill_between([Tmin,Tmax],[NN3sexp-NN3sexpdev, NN3sexp-NN3sexpdev],[NN3sexp+NN3sexpdev, NN3sexp+NN3sexpdev], alpha = 0.2,label = r'$c_{3\star}^{E}$')
        plt.xlim([temperatures_plots[0][0], 40])
        plt.ylim([-0.5, 1])
        plt.yticks(np.arange(-0.5,1,0.1))
        plt.legend(loc = 'lower left')
        plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Zoom" + addsave +"rmexpmag={0}.png".format(rmexpmag))

In [ ]:
np.savetxt('Temperatures.txt', temperatures_plots[0])
np.savetxt('Correlations.txt', t_h_MeanFc[0][:,0,:])

In [ ]:
for i in range(n):
    addsave = "L={0}".format(L[i]) + "J2ovJ1" +"={0}".format(J2[i]/J1[i]) + "J3povJ1" +"={0}".format(J3[i]/J1[i])  + "J3sovJ1" +"={0}".format(J3st[i]/J1[i])
    addtitle = r"$L$={0} ".format(L[i])+r"$J_2/J_1$={0}".format(J2[i]/J1[i]) + r"  $J_{3||}/J_1$" + "={0}".format(J3[i]/J1[i]) +r"  $J_{3\star}/J_1$"+"={0}".format(J3st[i]/J1[i])
    
    figsize=(11,9)
    dpi = 200
    tmin = 0
    tmax = 31
    plt.figure(figsize=figsize, dpi = dpi)
    for hid,h in enumerate(hfields_plots[0]):
        plt.semilogx(temperatures_plots[i][tmin:tmax],t_h_MeanFc[i][tmin:tmax,hid,1]-t_h_MeanFc[i][tmin:tmax,hid,2],label = r'$h/J_1$={0}'.format(h))
        plt.fill_between(temperatures_plots[i][tmin:tmax],
                        t_h_MeanFc[i][tmin:tmax,hid,1]-t_h_MeanFc[i][tmin:tmax,hid,2]-2*np.sqrt(t_h_varMeanFc[i][tmin:tmax,hid,2]),
                        t_h_MeanFc[i][tmin:tmax,hid,1]-t_h_MeanFc[i][tmin:tmax,hid,2]+2*np.sqrt(t_h_varMeanFc[i][tmin:tmax,hid,2]), alpha = 0.2)
    plt.title(addtitle)
    plt.xlabel(r"$T/J_1$")
    plt.ylabel(r"$\Delta(\langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle)$")
    plt.grid(which='both')
    plt.legend()

In [ ]:
 t_h_MeanFrustratedTriangles,t_h_varMeanFrustratedTriangles = \
    af.LoadFrustratedTriangles(foldername, filenamelist,listfunctions,
                               stat_temps, stat_hfields, nb, **kw)

In [ ]:
np.linspace(0,4.2,22)

In [ ]:
af.BasicPlotsTriangles(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanFrustratedTriangles,
                        t_h_varMeanFrustratedTriangles, **kw)
plt.plot([1e-3, 40],[0.116, 0.116])
plt.fill_between([1e-3,40],[0.09,0.09],[0.141,0.141],alpha = 0.2)

In [ ]:
import importlib
importlib.reload(af)
import DualwormFunctions as dw

In [ ]:
ids2walker = idswalker

In [ ]:
hid = 1

In [ ]:
print(hfields[0][hid])

In [ ]:
# Plot the lowest temperature state to get an idea
t_spinstates, t_states, t_charges = af.LoadStates(foldername, filenamelist, L, nh)
latsize = t_spinstates[0][hid].size
[d_ijl, ijl_d, s_ijl, ijl_s, d_2s, s2_d, d_nd, d_vd, d_wn,
     sidlist, didlist, c_ijl, ijl_c, c2s, csign] =\
    dw.latticeinit(L[0])
print(ids2walker[0][0,hid])

In [ ]:
tid = 2
Lp = L[0]
state = t_states[0][ids2walker[0][tid,hid]]
spinstate = t_spinstates[0][ids2walker[0][2,hid]]
#chargestate = t_charges[0][ids2walker[0][0,0]]

In [ ]:
kdraw.plotstate(0, Lp, d_ijl, ijl_s, sidlist, didlist, s2_d, [state], [spinstate],
                dim_node_color = 'orange', dim_color ='orange', 
                spin_up_color = 'blue', spin_down_color = 'red')
#kdraw.plot_honeycomb_chargestate(chargestate, ijl_c,
#                                 L,2,c1_color = 'black', c2_color = 'green',
#                                 c3_color = 'red', c4_color = 'black', uponly=True)
plt.legend()
kdraw.plotstate(0, Lp, d_ijl, ijl_s, sidlist, didlist, s2_d, [state], [spinstate],
                dim_node_color = 'none', dim_color ='none', 
                spin_up_color = 'blue', spin_down_color = 'red')
#kdraw.plot_honeycomb_chargestate(chargestate, ijl_c,
#                                 L,2,c1_color = 'black', c2_color = 'green',
#                                 c3_color = 'red', c4_color = 'black', uponly=True)
plt.legend()

In [ ]:
ijl_s[(L[0],L[0],0)]

In [ ]:
# checking the energy:
config = af.correlationsTester(state, latsize, d_ijl, ijl_d, L[0])
m = obs.magnetisation(len(s_ijl), state, [],spinstate, s_ijl, ijl_s)
H = config['J1']*J1[0] + config['J2']*J2[0] + config['J3']*J3[0]+ config['J3st']*J3st[0] - m*hfields[0][hid]
print("Former energy: ", H)

In [ ]:
config

In [ ]:
m

In [ ]:
kdraw.plotstate(0, Lp, d_ijl, ijl_s, sidlist, didlist, s2_d, [state], [spinstate],
                dim_node_color = 'orange', dim_color ='orange', 
                spin_up_color = 'blue', spin_down_color = 'red')
# spot the two spins that need to be flipped
listspins = [ijl_s[(L[0],L[0],0)],ijl_s[(L[0]+1,L[0]-1,2)]]
kdraw.plot_kag_spins(listspins, ijl_s, L[0], 2, color = 'green')
listdimers = [ijl_d[(L[0], L[0], 0)],
                        ijl_d[(L[0]+1, L[0]-1, 4)],
                        ijl_d[(L[0]+1, L[0]-1, 3)],
                        ijl_d[(L[0]+1, L[0], 5)],
                        ijl_d[(L[0]+1, L[0], 4)],
                        ijl_d[(L[0], L[0], 2)]
                       ]
kdraw.plot_dice_dimers(listdimers, d_ijl, L[0], 2, color = 'green')

newspinstate = np.copy(spinstate); # otherwise messing spinstate up
for spin in listspins:
    newspinstate[spin] = - newspinstate[spin]
# update the dimer state accordingly, because can be read much more easily.
newstate = np.copy(state);
for dimer in listdimers:
    newstate[dimer] = - newstate[dimer]

plt.legend()

plt.figure()
kdraw.plotstate(0, Lp, d_ijl, ijl_s, sidlist, didlist, s2_d, [newstate], [newspinstate],
                dim_node_color = 'none', dim_color ='none', 
                spin_up_color = 'blue', spin_down_color = 'red')
plt.figure()
kdraw.plotstate(0, Lp, d_ijl, ijl_s, sidlist, didlist, s2_d, [newstate], [newspinstate],
                dim_node_color = 'orange', dim_color ='orange', 
                spin_up_color = 'blue', spin_down_color = 'red')

# Compute the energy of this new configuration
config = af.correlationsTester(newstate, latsize, d_ijl, ijl_d, L[0])
m = obs.magnetisation(len(s_ijl), newstate, [], newspinstate, s_ijl, ijl_s)
H = config['J1']*J1[0] + config['J2']*J2[0] + config['J3']*J3[0]+ config['J3st']*J3st[0] - m*hfields[0][hid]
print("New energy: ", H)

In [ ]:
config

In [ ]:
m